In [340]:
import numpy as np
from Crypto.Util.number import getPrime, inverse
from hashlib import sha256
from random import randint
from Crypto.PublicKey import RSA
import math

# Key Generation
1. Choose two large primes $p$ and $q$
2. Compute the modulus $n = p \cdot q$
3. Compute the number of integers that are coprime to $n$ using Euler's phi (totient) function: $\phi(n) = (p-1)(q-1)$
4. Select public exponent $e \in \{1,2,...,\phi(n) -1\}$ such that
$$gcd(e,\phi(n)) = 1$$
5. Compute private key $d$ such that
$$d \cdot e \equiv \mod \phi(n)$$

In [341]:
def generate_keypair(bits: int = 2048): 
    """
    Generate an RSA key pair consisting of a public key and a private key.
    """
    p = getPrime(bits // 2)
    q = getPrime(bits // 2)
    while p == q: #ensures p != q
        p = getPrime(bits // 2)
    n = p * q #calculate modulus
    phi_n = (p - 1) * (q - 1) #Euler's phi function

    e = 65537 #standard value for public exponent e
    if math.gcd(e, phi_n) != 1: 
        return generate_keypair(bits)

    d = inverse(e, phi_n) #private exponent d

    public_key = (n, e)
    private_key = (n, d)

    return public_key, private_key

# Key terms
1. msg := plaintext message encoded with utf-8
2. x := msg converted to bytes
3. y := encrypted message x

# Encryption 
$$y = e_{k_{receiver,pub}}(x) \equiv x^{e} \mod n$$

In [342]:
def encrypt_message(message: bytes, public_key) -> int: 
    """
    Encrypt encoded message x using receiver's public key.

    Output: ciphertext y (int) 
    """
    n, e = public_key
    x = int.from_bytes(message, 'big')
    y = pow(x, e, n)
    
    return y

# Signature

In [343]:
def signature(message: bytes, private_key) -> int: 
    """
    Sign the hash digest of the encrypted message with the sender's private key
    """
    n, d = private_key
    h = sha256(message).digest() #get hash digest
    h_int = int.from_bytes(h, 'big')
    sign = pow(h_int, d, n)

    return sign

### Here the sender sends both the signature $sign$ and the ciphertext $y$

# Decryption
$$x = d_{k_{receiver,priv}}(y) \equiv y^{d} \mod n$$

In [344]:
def decrypt_message(ciphertext_y: int, private_key) -> str: 
    """
    Decrypt ciphertext y with receiver's private key

    Output: plaintext bytes and plaintext
    """
    n, d = private_key
    x = pow(ciphertext_y,d, n)
    plaintext_bytes = x.to_bytes((x.bit_length() + 7) // 8, 'big')
    plaintext = int.from_bytes(plaintext_bytes)

    return plaintext_bytes, plaintext

# Verification

In [345]:
def verification(signature: int, public_key, message: bytes) -> bool:
    """
    Verifies the authenticity of the sender of the message. 
    Uses the sender's public key.
    
    Output: True or False (boolean)
    """
    n, e = public_key
    decrypt_to_hash = pow(signature, e, n)
    h = sha256(message).digest()
    h_int = int.from_bytes(h, 'big')
    
    return h_int == decrypt_to_hash

# Simulation Functions

In [346]:
def simulation_sender(message, receiver_pubkey: tuple, 
                      sender_privkey: tuple, sender_pubkey: tuple):
    #get ciphertext
    y = encrypt_message(message, receiver_pubkey)

    #get signature
    sign = signature(message, sender_privkey)

    #sender sends ciphertext (y) and signature (sign)
    return y, sign, sender_pubkey

def simulation_receiver(sent_items, receiver_privkey: tuple):
    ciphertext, sign, sender_pubkey = sent_items

    #decryption
    plaintext_bytes, plaintext = decrypt_message(ciphertext, receiver_privkey)

    #verification
    verify = verification(sign, sender_pubkey, plaintext_bytes)

    return plaintext, verify

def simulation_eavesdropper(intercept_items, tampering_type="flip_bits", substitute_message=None, receiver_pubkey=None):
    """
    Example eavesdropper that tampers with ciphertext

    Types of tampering:
        - "flip_bits_all": flip all bits of ciphertext bytes 
        - "flip_bits_random" : flips bits in a random position
        - "substitution": Replace ciphertext with encryption of a different message

    """
    ciphertext, signature, sender_pubkey = intercept_items

    if tampering_type == 'flip_bits_all':
        tampered_ciphertext = ciphertext ^ 1 #XOR 1 to flip bits
    
    if tampering_type == 'flip_bits_random':
        n = ciphertext.bit_length() #bit length
        if n == 0: #edge case: if ciphertext = 0, flip to 1
            tampered_ciphertext = 1
        else:
            k = randint(0, n - 1) #random position to flip bit
            #(n - 1) because the tampered integer will become larger at exactly n
            
            tampered_ciphertext = ciphertext ^ (1 << k) 
            #XOR ciphertext to flip the bits at position k (from the right)

    if tampering_type == 'substitution': #attacker chooses message
        if receiver_pubkey == None:
            raise ValueError("Receiver public key needed for substitution") 
        tampered_ciphertext = encrypt_message(substitute_message, receiver_pubkey)
        
    return (tampered_ciphertext, signature, sender_pubkey)


# Demo

In [347]:
def demo():
    #Generate random message
    message = randint(1, 2**128).to_bytes(16,"big")
    print(f'Randomly chosen plaintext: {int.from_bytes(message)}\n')

    #Generating keys
    sender_public, sender_private = generate_keypair()
    receiver_public, receiver_private = generate_keypair()

    #Simulate communication with NO EAVESDROPPER
    sent_items = simulation_sender(message, 
                                   receiver_public, 
                                   sender_private, 
                                   sender_public) 
    encrypted_message, signature, sender_public = sent_items
        #Sender sends encrypted message, signature and sender public key

    recovered_plaintext, verify_status = simulation_receiver(sent_items, receiver_private)
    print(f'No eavesdropper ->\nRecovered plaintext: {recovered_plaintext}\nSender verified?: {verify_status}\n')

    #Simulate bit flipping
    intercepted_items = sent_items
    tampered_items = simulation_eavesdropper(intercepted_items,tampering_type='flip_bits_all')
    recovered_plaintext, verify_status = simulation_receiver(tampered_items, receiver_private)
    print(f'Entire message bits flipped->\nRecovered plaintext: {recovered_plaintext}\nSender verified?: {verify_status}\n')

    #Simulate one random bit flipping
    intercepted_items = sent_items
    tampered_items = simulation_eavesdropper(intercepted_items,tampering_type='flip_bits_random')
    recovered_plaintext, verify_status = simulation_receiver(tampered_items, receiver_private)
    print(f'One bit randomly flipped->\nRecovered plaintext: {recovered_plaintext}\nSender verified?: {verify_status}\n')

    #Simulate entire message replaced
    replacement_message = randint(1, 2**128).to_bytes(16,"big")
    tampered_items = simulation_eavesdropper(intercepted_items,tampering_type='substitution', 
                                             substitute_message=replacement_message, receiver_pubkey=receiver_public)
    recovered_plaintext, verify_status = simulation_receiver(tampered_items, receiver_private)
    print(f'Entire message replaced->\nRecovered plaintext: {recovered_plaintext}\nSender verified?: {verify_status}\n')

demo()

Randomly chosen plaintext: 51084947579396079360995391408893224672

No eavesdropper ->
Recovered plaintext: 51084947579396079360995391408893224672
Sender verified?: True

Entire message bits flipped->
Recovered plaintext: 6850849766422799884117332439000647167608920513816996848708043004567650676434766831234847422709906968258808174811611916514268636718553869813199023155139695803925745622168777342717656017843660870598003097994241650043116515457989315090723909195208412119974148892819303138638994464570945039487172225852768441749529669808809237175769829482474909823639376601449480024141525540002358799104696471998906013252421514141202622865640397425472206466293258295414007230986251842267793327720491862945463989091783177689108403698368877786495679204992016234155193208535870218063643385312710844625864831464579173564773537057555089591301
Sender verified?: False

One bit randomly flipped->
Recovered plaintext: 87536368674564111446672256206948386137021696042035511057875311036540625021840732253852094